In [25]:
from src.controller.HwpController import HwpController

import re, uuid
import numpy as np

In [39]:
import re, uuid
import numpy as np

from src.service.ParseService import ParseService


class SimpleParser:
    def delete_non_target_data(self, table_data):
        """
        한글 표에서 원하지 않은 부분까지 나온 데이터를 정리하여 리스트로 반환합니다.
        target_data_text에 표에 반복적으로 들어가는 텍스트를 입력하여 필요없는 데이터를 삭제합니다.
        """
        target_data_text = ['구분', '진동속도', '진동레벨', '소음', '허용기준', '비고']

        target_data = [
            sublist for sublist in table_data
            if any(
                entry['row'] in ['1', '2'] and any(keyword in entry['text'] for keyword in target_data_text)
                for entry in sublist
            )
        ]

        target_data = [[item for item in items if item['text'] != ''] for items in target_data]

        return target_data

    def extract_columns(self, table_list):
        """
        한글 표에서 공통적인 컬럼 부분을 추출합니다.
        컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
        또한 한글 표를 row순서대로 읽기 때문에 특정 병합이 되어 있을 경우
        한글에선 뒤에 있는 컬럼이 앞에 있을 수 있기 때문에 해당 부분도 max함수와 row값을 이용해서 조정합니다.
        이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
        """
        columns = []
        for items in table_list:
            for item in items:
                if item not in [i for i in columns]:
                    row = int(item['row'])

                    if row == 0 and row == 0:
                        continue
                    elif row == 1 or row == 2:
                        columns.append(item)
        
        data = {
            'row': 0,
            'col': 0,
            'colspan': 1,
            'rowspan': 1,
            'text': '일시'
        }
        columns.insert(0, data)

        return columns
    
    def extract_non_column_data(self, table_list, columns):
        """
        한글 표에서 컬럼 부분을 제거한 나머지 데이터들을 반환합니다.
        """
        dict_list = []
        for items in table_list:
            data = []
            for item in items:
                if not int(item['colspan']) > 1 and item['text'] not in [column['text'] for column in columns]:
                    data.append(item)
            dict_list.append(data)

        return dict_list

    def group_by_date(self, dict_list):
        """
        한글 표 데이터를 날짜 별로 분류하여 리스트로 저장하여 반환합니다.
        날짜 별 분류는 하나의 TableCell에 포함된 동일한 row값들 끼리 묶는 것으로 수행합니다.
        """
        group_list = []

        for items in dict_list:
            rows = list(set([int(item['row']) for item in items]))
            for row in rows:
                print(row)
                temp = [item for item in items if int(item['row']) == row]
                group_list.append(temp)
        
        return group_list
    
    def update_merge_data(self, group_list):
        """
        한글 표에 병합 처리된 셀에 대한 데이터 처리를 완료한 뒤 리스트로 반환합니다.
        SimpleParser에서는 병합 셀이 아닌 셀의 최초 값이 공통 값으로 결정되기 때문에 이를 통해 값을 추가합니다.
        """
        cached_head_data = []

        for idx, items in enumerate(group_list):
            if len(items) == 1:
                cached_head_data = items

            new_items = []
            if any(data not in items for data in cached_head_data):
                new_items.extend(cached_head_data)

            group_list[idx] = new_items + items

        return [items for items in group_list if len(items) != 1]

    
    def serialize_to_dict(self, group_list, columns):
        """
        컬럼 리스트와 파싱이 끝난 그룹 리스트를 이용해서 데이터를 분류한 뒤 리스트로 반환합니다.
        컬럼 리스트에 대응하는 값들을 그룹 리스트에서 찾아서 추가해주는 작업을 수행합니다.
        """
        serialize_list = []
        cached_columns = []
        
        for item in columns:
            if int(item['colspan']) > 1:
                for t in ['최저치', '최고치']:
                    data = {
                        'row': item['row'],
                        'col': item['col'],
                        'colspan': item['colspan'],
                        'rowspan': item['rowspan'],
                        'text': f'{item['text']} {t}'
                    }
                    cached_columns.append(data)
            else:
                cached_columns.append(item)

        for items in group_list:
            data = {column['text']: item['text'] for column, item in zip(cached_columns, items) if column != '발파횟수'}
            serialize_list.append(data)

        return serialize_list


In [40]:
controller = HwpController()
service = SimpleParser()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(김해상동물류)_간단이.hwp'

controller.get_table_list(path, '일자별 계측 현황')
table_list = service.delete_non_target_data(controller.get_list())
table_list = 

table_list

[[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '3월4일'},
  {'row': '1', 'col': '0', 'colspan': '1', 'rowspan': '2', 'text': '구분'},
  {'row': '1',
   'col': '1',
   'colspan': '2',
   'rowspan': '1',
   'text': '진동속도(cm/s)'},
  {'row': '1',
   'col': '3',
   'colspan': '2',
   'rowspan': '1',
   'text': '진동레벨[dB(V)]'},
  {'row': '1',
   'col': '5',
   'colspan': '2',
   'rowspan': '1',
   'text': '소음[dB(A)]'},
  {'row': '1', 'col': '7', 'colspan': '1', 'rowspan': '2', 'text': '허용기준'},
  {'row': '1', 'col': '8', 'colspan': '1', 'rowspan': '2', 'text': '비고'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '최고치'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '최고치'},
  {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
  {'row': '2', 'col': '6', 'colspan': '1', 

In [41]:
columns = service.extract_columns(table_list)

columns

[{'row': 0, 'col': 0, 'colspan': 1, 'rowspan': 1, 'text': '일시'},
 {'row': '1', 'col': '0', 'colspan': '1', 'rowspan': '2', 'text': '구분'},
 {'row': '1',
  'col': '1',
  'colspan': '2',
  'rowspan': '1',
  'text': '진동속도(cm/s)'},
 {'row': '1',
  'col': '3',
  'colspan': '2',
  'rowspan': '1',
  'text': '진동레벨[dB(V)]'},
 {'row': '1', 'col': '5', 'colspan': '2', 'rowspan': '1', 'text': '소음[dB(A)]'},
 {'row': '1', 'col': '7', 'colspan': '1', 'rowspan': '2', 'text': '허용기준'},
 {'row': '1', 'col': '8', 'colspan': '1', 'rowspan': '2', 'text': '비고'},
 {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
 {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '최고치'},
 {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
 {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '최고치'},
 {'row': '2', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '최저치'},
 {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': '최고치'}]

In [42]:
dict_list = service.extract_non_column_data(table_list, columns)

dict_list

[[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '3월4일'},
  {'row': '3', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '대감터널'},
  {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '1.0cm/s'},
  {'row': '3', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'O.K'},
  {'row': '4', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '애니시스템'},
  {'row': '4', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '0.021'},
  {'row': '4', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '0.027'},
  {'row': '4'

In [43]:
group_list = service.update_merge_data(service.group_by_date(dict_list))

group_list


0
3
4
5
8
9
10
13
14
15
18
19
0
3
4
5
8
9
10
13
14
15
18
19
20
23
24
0
3
4
5
8
9
10
13
14
15
18
19
20
23
24
0
3
4
5
8
9
10
13
14


[[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '3월4일'},
  {'row': '3', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '대감터널'},
  {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '3', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': '1.0cm/s'},
  {'row': '3', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'O.K'}],
 [{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '3월4일'},
  {'row': '4', 'col': '0', 'colspan': '1', 'rowspan': '1', 'text': '애니시스템'},
  {'row': '4', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '0.021'},
  {'row': '4'

In [44]:
serialize_list = service.serialize_to_dict(group_list, columns)

serialize_list

[{'일시': '3월4일',
  '구분': '대감터널',
  '진동속도(cm/s) 최저치': 'N/T',
  '진동속도(cm/s) 최고치': 'N/T',
  '진동레벨[dB(V)] 최저치': 'N/T',
  '진동레벨[dB(V)] 최고치': 'N/T',
  '소음[dB(A)] 최저치': 'N/T',
  '소음[dB(A)] 최고치': 'N/T',
  '허용기준': '1.0cm/s',
  '비고': 'O.K'},
 {'일시': '3월4일',
  '구분': '애니시스템',
  '진동속도(cm/s) 최저치': '0.021',
  '진동속도(cm/s) 최고치': '0.027',
  '진동레벨[dB(V)] 최저치': '47.55',
  '진동레벨[dB(V)] 최고치': '53.42',
  '소음[dB(A)] 최저치': '18.24',
  '소음[dB(A)] 최고치': '84.55',
  '허용기준': '0.2cm/s75dB(V)75dB(A)',
  '비고': 'O.K'},
 {'일시': '3월6일',
  '구분': '대감터널',
  '진동속도(cm/s) 최저치': '0.125',
  '진동속도(cm/s) 최고치': '0.375',
  '진동레벨[dB(V)] 최저치': '-',
  '진동레벨[dB(V)] 최고치': '-',
  '소음[dB(A)] 최저치': '-',
  '소음[dB(A)] 최고치': '-',
  '허용기준': '1.0cm/s',
  '비고': 'O.K'},
 {'일시': '3월6일',
  '구분': 'GS물류센터',
  '진동속도(cm/s) 최저치': '0.039',
  '진동속도(cm/s) 최고치': '0.057',
  '진동레벨[dB(V)] 최저치': '54.6',
  '진동레벨[dB(V)] 최고치': '58.83',
  '소음[dB(A)] 최저치': '57.96',
  '소음[dB(A)] 최고치': '63.62',
  '허용기준': '0.2cm/s75dB(V)75dB(A)',
  '비고': 'O.K'},
 {'일시': '3월7일',
  '구분': '대

In [45]:
import pandas as pd

# pandas DataFrame으로 변환
df = pd.DataFrame(serialize_list)

# 계측 위치별로 데이터 프레임 분류
locations = df['구분'].unique()  # 계측 위치별 고유값을 찾음

# 각 위치별 데이터 프레임을 딕셔너리에 저장
dfs = {}
for location in locations:
    dfs[location] = df[df['구분'] == location]

# 결과 확인
# 예시: 과천지식정보타운 위치의 데이터 프레임 출력
# print(dfs['GS물류센터'])
print(dfs['대감터널'])

       일시    구분 진동속도(cm/s) 최저치 진동속도(cm/s) 최고치 진동레벨[dB(V)] 최저치 진동레벨[dB(V)] 최고치  \
0    3월4일  대감터널            N/T            N/T             N/T             N/T   
2    3월6일  대감터널          0.125          0.375               -               -   
4    3월7일  대감터널          0.146           0.24               -               -   
6    3월9일  대감터널          0.088          0.164               -               -   
8   3월11일  대감터널              -          0.062               -               -   
10  3월12일  대감터널          0.042          0.176               -               -   
12  3월13일  대감터널          0.036          0.104               -               -   
14  3월14일  대감터널           0.11          0.359               -               -   
16  3월15일  대감터널          0.125          0.354               -               -   
18  3월16일  대감터널          0.081           0.16               -               -   
20  3월20일  대감터널          0.056          0.093               -               -   
22  3월21일  대감터널          0.0